<a href="https://colab.research.google.com/github/cemvardar/pragmatic_google_colab/blob/main/dslab_colab_utility_shopify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import userdata
import requests

In [3]:
class Product:
    def __init__(self, shopify_data):
        self.shopify_data = shopify_data
        self.title = self.shopify_data['title']
        self.product_type = self.shopify_data['product_type']
        self.body_html = self.shopify_data['body_html']
        self.tags = self.shopify_data['tags']
        self.id = self.shopify_data['id']
        self.handle = f"https://www.liplips.co.uk/products/{self.shopify_data['handle']}"
        self.status = self.shopify_data['status']
        self.is_active = self.status=='active'


class Collection():
    def __init__(self, shopify_data):
        self.shopify_data = shopify_data
        self.title = self.shopify_data['title']
        self.body_html = self.shopify_data['body_html']
        self.id = self.shopify_data['id']
        # self.collects = get_shopify_data('collects', ('collection_id', self.id))['collects']
        self.collects = None
        self.handle = f"https://www.liplips.co.uk/collections/{self.shopify_data['handle']}"
        self.admin_handle = f"https://admin.shopify.com/store/liplipsstore/collections/{self.id}"

    def products_list(self):
        products_list = []
        if self.collects == None:
            self.collects = get_shopify_data('collects', ('collection_id', self.id))['collects']
        for collect in self.collects:
            products_list.append(products_dict[collect['product_id']])
        return products_list

    def get_product_image_url(self, product_index):
        products = self.products_list()
        return products[product_index].shopify_data['images'][0]['src']

    def get_video_file_name(self):
        return f"{self.title.replace(' ', '_')}.mp4"


def get_api_url(element, query_tuple=None):
    url_template = f'https://liplipsstore.myshopify.com/admin/api/2024-04/{element}.json'
    if query_tuple is not None:
        query_suffix = f'?{query_tuple[0]}={query_tuple[1]}'
        return url_template+query_suffix
    return url_template

def get_shopify_data(element, query_tuple=None):
    access_token = userdata.get('liplips_shopify_token')
    shop_url = 'your_shop_name.myshopify.com'

    # Set up the headers with the necessary authentication
    headers = {
        "Content-Type": "application/json",
        "X-Shopify-Access-Token": access_token
    }
    params = {'limit': 250}
    # Define the endpoint for fetching products
    url = get_api_url(element, query_tuple)

    # Make the HTTP GET request to the Shopify API
    response = requests.get(url, headers=headers, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Load the JSON data from the response
        products = response.json()

    else:
        print(f"Failed to retrieve products: {response.status_code}")
    return products

def get_collection_dict():
    element = 'custom_collections'
    collections = get_shopify_data(element)[element]
    print(f'{len(collections)} collections')
    collection_dict = {}
    for collection_data in collections:
        collection = Collection(collection_data)
        collection_dict[collection.id] = collection

    return collection_dict


def get_product_dict():
    element = 'products'
    products = get_shopify_data(element)[element]

    print(f'{len(products)} products')

    products_dict = {}
    for product_data in products:
        product = Product(product_data)
        products_dict[product.id] = product

    return products_dict

products_dict = get_product_dict()
collection_dict = get_collection_dict()

154 products
87 collections


In [4]:
def add_product_to_shopify(product_data):
    # Headers for authentication and content type
    headers = {
        "Content-Type": "application/json",
        "X-Shopify-Access-Token": userdata.get('liplips_shopify_token')
    }

    # Data to be updated
    data = {
        "product": product_data
    }

    # API endpoint to update the product
    url = f'https://liplipsstore.myshopify.com/admin/api/2023-01/products.json'

    # Make the PUT request to update the product description
    response = requests.post(url, headers=headers, json=data)

    # Check the response
    if response.status_code == 201:
        print("Product description updated successfully.")
        updated_product = response.json()
        print(f'''https://admin.shopify.com/store/liplipsstore/products/{updated_product['product']['id']}''')
    else:
        print(f"Failed to update product: {response.status_code} - {response.text}")
    return updated_product

# product_data = {
#             "title": "aaaa test",
#             "vendor": "Liplips",
#             "body_html": 'testing',
#             "product_type": "Necklaces",
#             'status': 'draft',
#             "variants": [
#                 {
#                     "option1": "Default Title",
#                     "price": 10,
#                     "sku": 'rs_test',
#                     "inventory_management": "shopify",
#                     "inventory_quantity": 5
#                 }
#                 ],
#             'images': []
#             }

# updated_product = add_product_to_shopify(product_data)


Product description updated successfully.
https://admin.shopify.com/store/liplipsstore/products/7409814143018
